In [1]:
import re
import json
import pandas as pd

In [2]:
PATH = '/mnt/disk_d/hack-fu/data/raw/'
OUTPUT_PATH =  '/mnt/disk_d/hack-fu/data/interim/'

# train.csv

In [3]:
df_train = pd.read_csv(PATH+'train.csv')
df_train.head(3)

,index,name,description,target
0,324865089,продавец кассир,<strong>Обязанности:</strong> <ul> <li>работа ...,-1
1,169467135,продавец мила (шевченко 17),<p><strong>Магазин МИЛА по адресу б-р Шевченко...,-1
2,169939030,кассир в пиццерию г витебск,"<p><strong>Устал искать работу? Может, хочешь ...",5223


In [4]:
df_train_labeled = df_train[df_train['target'] != -1]
df_train_unlabeled = df_train[df_train['target'] == -1]

In [5]:
df_train_labeled['target'].value_counts().sort_values(ascending = False)

6121    1521
5223    1288
9613     622
2411     562
3221     500
        ... 
3252      30
4224      29
6111      29
1321      24
5246       1
Name: target, Length: 76, dtype: int64

# test.csv

In [6]:
df_test = pd.read_csv(PATH+'test.csv')
df_test.head(3)

,index,name,description
0,28357560,продавец консультант в магазин профессионально...,<strong>Обязанности:</strong> <ul> <li>Продажи...
1,114041896,торговый представитель (севастополь),<p><strong>Обязанности:</strong></p> <ul> <li>...
2,78645675,торговый представитель (город+район заречный),<p><strong>Ключевые задачи:</strong></p> <ul> ...


# okz_3_4_professions.csv

In [7]:
#df_okz = pd.read_csv(PATH+'okz_3_4_professions.csv')
#df_okz.head(3)

# parsed_json.json

In [8]:
df_json = pd.read_json(PATH+'vacancy_descriptions/1_parsed.json')
df_json.head(3)

,ID,Position,Content
0,363107946,Секретарь,"{'Условия': ['Срочно требуется:Секретарь', 'Вы..."
1,363125198,Заместитель управляющего магазином,{'Условия': ['Наша миссия — мы делаем качество...
2,363144355,Персональный менеджер,"{'Требования': ['окончили вуз', 'разбираетесь ..."


In [9]:
json_paths = [PATH+'vacancy_descriptions/'+str(i)+'_parsed.json' for i in range(2,6)]
json_paths

['/mnt/disk_d/hack-fu/data/raw/vacancy_descriptions/2_parsed.json',
 '/mnt/disk_d/hack-fu/data/raw/vacancy_descriptions/3_parsed.json',
 '/mnt/disk_d/hack-fu/data/raw/vacancy_descriptions/4_parsed.json',
 '/mnt/disk_d/hack-fu/data/raw/vacancy_descriptions/5_parsed.json']

In [10]:
for path in json_paths:
    df_json_temp = pd.read_json(path)
    df_json = pd.concat([df_json,df_json_temp],axis = 0)

In [11]:
df_json['content'] = df_json['Content'].apply(lambda x: list(x.values()))

In [12]:
df_json['index'] = df_json['ID'].rename('index') 

In [13]:
df_json = df_json.drop(['Content','ID'],axis = 1)

In [14]:
df_json

,Position,content,index
0,Секретарь,"[[Срочно требуется:Секретарь, Выплаты ежедневн...",363107946
1,Заместитель управляющего магазином,[[Наша миссия — мы делаем качество доступным! ...,363125198
2,Персональный менеджер,"[[окончили вуз, разбираетесь в законах о банко...",363144355
3,Продавец-консультант ECCO (ТЦ Сити Молл),[[Обладаешь хорошими коммуникативными навыками...,363340232
4,Агент/Риэлтор (Помощник Риэлтора),[[Организовывать и проводить просмотры/показы ...,363383054
...,...,...,...
7560,Подсобный рабочий,[[В сведениях о потребности в работниках работ...,38512229
7561,Электросварщик на автоматических и полуавтомат...,"[[ЭЛЕКТРОСВАРКА ДЕТАЛЕЙ И УЗЛОВ,ВЫЯВЛЕНИЕ И П...",38512271
7562,Электромонтер оперативно-выездной бригады 3 ра...,[[Выполняет должностные обязанности согласно д...,38512900
7563,Оператор машинного доения,[[Выполнение должностных обязанностей в соотве...,38513191


# Merging

In [15]:
df = df_json.merge(df_train,how='left',on='index')
df.shape

(47566, 6)

In [16]:
df

,Position,content,index,name,description,target
0,Секретарь,"[[Срочно требуется:Секретарь, Выплаты ежедневн...",363107946,секретарь,<p><strong>Срочно требуется:</strong></p> <p>С...,-1.0
1,Заместитель управляющего магазином,[[Наша миссия — мы делаем качество доступным! ...,363125198,заместитель управляющего магазином,<p><strong>Что мы предлагаем:</strong></p> <p>...,1420.0
2,Персональный менеджер,"[[окончили вуз, разбираетесь в законах о банко...",363144355,NaN,NaN,NaN
3,Продавец-консультант ECCO (ТЦ Сити Молл),[[Обладаешь хорошими коммуникативными навыками...,363340232,продавец консультант ecco (тц сити молл),<p>Датская компания <strong>ECCO</strong> боле...,-1.0
4,Агент/Риэлтор (Помощник Риэлтора),[[Организовывать и проводить просмотры/показы ...,363383054,NaN,NaN,NaN
...,...,...,...,...,...,...
47561,Подсобный рабочий,[[В сведениях о потребности в работниках работ...,38512229,подсобный рабочий,В сведениях о потребности в работниках работод...,-1.0
47562,Электросварщик на автоматических и полуавтомат...,"[[ЭЛЕКТРОСВАРКА ДЕТАЛЕЙ И УЗЛОВ,ВЫЯВЛЕНИЕ И П...",38512271,электросварщик на автоматических и полуавтомат...,"ЭЛЕКТРОСВАРКА ДЕТАЛЕЙ И УЗЛОВ,ВЫЯВЛЕНИЕ И ПРЕ...",-1.0
47563,Электромонтер оперативно-выездной бригады 3 ра...,[[Выполняет должностные обязанности согласно д...,38512900,электромонтер оперативно выездной бригады 3 ра...,Выполняет должностные обязанности согласно дол...,7412.0
47564,Оператор машинного доения,[[Выполнение должностных обязанностей в соотве...,38513191,оператор машинного доения,Выполнение должностных обязанностей в соответс...,-1.0


In [17]:
df = df.drop('name',axis = 1)
df['target'] = df['target'].fillna(-1.0)
df['target'] = df['target'].astype(int)
df['description'] = df['description'].astype(str)
df.head(3)

,Position,content,index,description,target
0,Секретарь,"[[Срочно требуется:Секретарь, Выплаты ежедневн...",363107946,<p><strong>Срочно требуется:</strong></p> <p>С...,-1
1,Заместитель управляющего магазином,[[Наша миссия — мы делаем качество доступным! ...,363125198,<p><strong>Что мы предлагаем:</strong></p> <p>...,1420
2,Персональный менеджер,"[[окончили вуз, разбираетесь в законах о банко...",363144355,nan,-1


# Cleaning

In [18]:
html_regexp = re.compile(r'<.*?>')
df['description'] = df['description'].apply(lambda x:  html_regexp.sub(r'', x))

# Saving

In [19]:
df_labeled = df[df['target'] != -1].to_csv(OUTPUT_PATH+'labeled.csv',index = False)
df_unlabeled = df[df['target'] == -1].to_csv(OUTPUT_PATH+'unlabeled.csv',index = False)